# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
!pip install openai
from openai import OpenAI
from google.colab import userdata
#import os
# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv())

OPENAI_API_KEY = userdata.get('Ironhack-GPT')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00


In [2]:
#Function to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [8]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {
    'table': ['customers', 'orders', 'products', 'employees', 'reviews'],
    'definition':
        ['ID, name, contact information, and address.',
        'order ID, customer ID, product ID, order date, and status.',
        'product ID, name, description, price, and stock quantity.',
        'employee ID, name, position, and contact information.',
        'review ID, product ID, customer ID, rating, and review text.']
}
df = pd.DataFrame(data)
print(df)

       table                                         definition
0  customers        ID, name, contact information, and address.
1     orders  order ID, customer ID, product ID, order date,...
2   products  product ID, name, description, price, and stoc...
3  employees  employee ID, name, position, and contact infor...
4    reviews  review ID, product ID, customer ID, rating, an...


In [9]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [10]:
print(text_tables)

customers: ID, name, contact information, and address.
orders: order ID, customer ID, product ID, order date, and status.
products: product ID, name, description, price, and stock quantity.
employees: employee ID, name, position, and contact information.
reviews: review ID, product ID, customer ID, rating, and review text.


In [11]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [17]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question='where do i find the product ids?')

In [18]:
print(return_OAI(pqt1))

{
    "tables": ["products"]
}


In [29]:
pqt3 = prompt_question_tables.format(tables=text_tables, question='customer order and rating please')

In [30]:
print(return_OAI(pqt3))

{
    "tables": ["customers", "orders", "reviews"]
}


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?